# Librosa API
#### Objective
4. Build librosa class
    * Convert each preview in /data/audio to spectogram
    * Add spectograms to subfolder /data/spectograms
    * Create final dataframe (like with audio file conversion) that contains spectrogram paths
    

In [1]:
import sys
sys.path.append('../')
import os
import matplotlib.pyplot as plt

#for loading and visualizing audio files
import librosa
import librosa.display

#To load csvs and adjust arrays
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

In [2]:
data = pd.read_csv('../data/tracklist_mp3link.csv', sep=";")
data.head()

,track_name,artist,album,track_url,preview_url,artist_url,genres,file_location
0,Guillotine,Stray From The Path,Guillotine,https://api.spotify.com/v1/tracks/5mWs0zabZrRE...,https://p.scdn.co/mp3-preview/dbd5861d09ee540c...,https://api.spotify.com/v1/artists/5tGG1slV9pk...,"['deathcore', 'long island punk', 'melodic har...",../data/audio/Guillotine-Stray From The Path.wav
1,Vermin,Allegaeon,DAMNUM,https://api.spotify.com/v1/tracks/6FDLIsQnjDx5...,https://p.scdn.co/mp3-preview/7d8fba2941244a72...,https://api.spotify.com/v1/artists/5NIvyOvy39l...,"['american melodeath', 'denver metal', 'melodi...",../data/audio/Vermin-Allegaeon.wav
2,Put Your Back Into The Oar,Amon Amarth,Put Your Back Into The Oar,https://api.spotify.com/v1/tracks/4fY2JR0REUiy...,https://p.scdn.co/mp3-preview/d2253d713457b378...,https://api.spotify.com/v1/artists/3pulcT2wt7F...,"['death metal', 'melodic death metal', 'metal'...",../data/audio/Put Your Back Into The Oar-Amon ...
3,Make Believe,Memphis May Fire,Make Believe,https://api.spotify.com/v1/tracks/5NLi9YVXt9pM...,https://p.scdn.co/mp3-preview/50d174c4df0bdff7...,https://api.spotify.com/v1/artists/7cNNNhdJDrt...,"['melodic metalcore', 'metalcore', 'post-screa...",../data/audio/Make Believe-Memphis May Fire.wav
4,Dystopia (feat. Christoph Wieczorek of Annisokay),Caliban,Dystopia (feat. Christoph Wieczorek of Annisokay),https://api.spotify.com/v1/tracks/2WaaniEgnI8p...,https://p.scdn.co/mp3-preview/927ee1c9965680c1...,https://api.spotify.com/v1/artists/1oeo6cC9Fqa...,"['deathcore', 'german metalcore', 'german post...",../data/audio/Dystopia feat Christoph Wieczore...


In [3]:
data.file_location.duplicated().sum()

21

In [4]:
data.shape

(710, 8)

In [5]:
data.drop_duplicates(subset=['file_location'], inplace=True)

In [6]:
data.shape

(689, 8)

In [7]:
data.reset_index(drop=True, inplace=True)

In [8]:
data['spectrogram'] = ''

In [14]:
def make_spectrograms(df):
    img_path = "../data/images/spectrograms/"
    
    for row in tqdm(range(len(df))):
#         audio_path = df['file_location'][row]
        track = df['track_name'][row]
        artist = df['artist'][row]
        spect = f"{track}_{artist}_spectro.png"

        path = df['file_location'][row]
        
        x, sr = librosa.core.load(path, mono=True, offset=0.0, duration=29, res_type='kaiser_best')
        
        print(path)
        # Convert waveform to spectrogram
        X = librosa.stft(x)
        Xdb = librosa.amplitude_to_db(abs(X))
        plt.figure(figsize=(14, 5))
        librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
        plt.savefig(img_path+spect)
        data.at[row, 'spectrogram'] = img_path+spect

# RUN ONLY ONCE
make_spectrograms(data)

In [18]:
data.to_csv("../data/tracklist_spectrograms.csv", sep=";", index=False)